## 分车型预测

In [1]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import plot_model
from keras.models import Sequential
from keras.engine.topology import Layer
from keras.layers import Dense, Dropout
from keras.initializers import glorot_uniform
from keras.layers import LSTM, RNN, GRU, SimpleRNN, Bidirectional, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.initializers import glorot_uniform

# from IPython.core.interactiveshell import InteractiveShell 
# InteractiveShell.ast_node_interactivity = 'all'    # 默认为'last'

seed = 2019
random.seed(seed)
tf.set_random_seed(seed)
np.random.seed(seed)
warnings.filterwarnings('ignore')

C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

In [2]:
data_path = './data/'
train_user_reply_data = pd.read_csv(data_path + 'train_user_reply_data.csv', encoding='utf-8')
train_search_data = pd.read_csv(data_path + 'train_search_data.csv', encoding='utf-8')
train_sales_data = pd.read_csv(data_path + 'train_sales_data.csv', encoding='utf-8')
evaluation_public = pd.read_csv(data_path + 'evaluation_public.csv', encoding='utf-8')

train_sales_data = train_sales_data.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True)
evaluation_public = evaluation_public.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True)

In [3]:
cars = ['a50a6db3d691e9ce', 'b135f55ced5c4c67', '9011c3cf495e3c7b', '28e29f2c03dcd84c', '54fc07138d70374c', 'd0f245b8781e3631', 'ffeb3a4cec627e8c', 'a7128d308ae14d73', '7aab7fca2470987e', '42d4439853da7221', '2509a94e62fd3e4b', '9e967965ad4a6c07', '346393c2c6305fb1', '7a7885e2d7c00bcf', '17bc272c93f19d56', 'f8a6975573af1b33', 'feabbf46658382b9', 'fde95ea242abd896', 'd4efbebb087fd03f', '04e66e578f653ab9', '7245e0ee27b195cd', 'f270f6a489c6a9d7', '02aab221aabc03b9', 'b4be3a4917289c82', 'f5d69960089c3614', 'c06a2a387c0ee510', '6858d6dfe680bdf7', '12f8b7e14947c34d', '07a30393ec51530e', 'cc21c7e91a3b5a0c', '3c974920a76ac9c1', '37aa9169b575ef79', 'ea489c253676aafc', '97f15de12cfabbd5', 'fc32b1a017b34efe', 'b25c4e2e3856af22', '854982e5264fb53e', '8527c11fccaa94e2', '283d331fdf05a655', '9a390098bf87b814', 'a28bb927b6fcb33c', 'a9a43d1a7ecbe75d', 'ef76a85c4b39f693', '4a103c30d593fbbe', '0797526c057dcf5b', 'c6cd4e0e073f5ac2', '1181894ae2811540', 'af6f4f548684e14d', '6155b214590c66e6', '5b1c11c3efed5312', '936168bd4850913d', 'c6833cb891626c17', '8cbac8dd8f0e89e2', 'a432c483b5beb856', 'cd5841d44fd7625e', '2a2ab41f8f6ff1cb', '7023efdab9cedc03', 'bb9fbec9a2833839', '79de4e4b24c35b04', '4f79773e600518a6', '28956642949e0e57', '3d7554f1f56dd664', 'b9aa8592ab2492da', '17363f08d683d52b', 'bc5bbd459c18fefd', '0aa15031db420212', '3e21824be728cbec', '7cf283430b3b5e38', '212083a9246d2fd3', 'da457d15788fe8ee', 'a207df29ec9583f0', '2d0d2c3403909fdb', '8c915fe4632fb9fa', '06880909932890ca', 'dff803b4024d261d', '63065128401bb3ff', '9c1c7ee8ebdda299', 'e8a33f68c15dcf7c', '32d3069d17aa47c2', '5f727f32393ade77', '5d7fb682edd0f937', '61e73e32ad101892']
provinces = ['浙江', '福建', '四川', '陕西', '安徽', '湖南', '广东', '云南', '上海', '山东', '湖北', '黑龙江', '江苏', '广西', '内蒙古', '辽宁', '北京', '重庆', '河北', '山西', '江西', '河南']

print(len(cars))
print(len(provinces))

# Normalization
train_sales_data['salesVolume'] = train_sales_data['salesVolume'].map(lambda index: np.log2(index) + 1)
scaler = MinMaxScaler(feature_range=(0, 1))
train_sales_data[['salesVolume']] = scaler.fit_transform(train_sales_data[['salesVolume']])
train_sales_data.head(3)

82
22


,province,adcode,model,bodyType,regYear,regMonth,salesVolume
506,上海,310000,02aab221aabc03b9,Sedan,2016,1,0.666707
507,云南,530000,02aab221aabc03b9,Sedan,2016,1,0.567856
508,内蒙古,150000,02aab221aabc03b9,Sedan,2016,1,0.469225


In [4]:
def generate_batch(data_set, look_back, gap_days):
    data_x, data_y = [], []
    for i in range(len(data_set) - look_back - gap_days + 1):
        a = data_set[i:(i + look_back), 0]
        data_x.append(a)
        data_y.append([data_set[i + look_back + gap_days - 1, 0]])
    return np.array(data_x), np.array(data_y), data_set[-look_back:, 0].reshape(1, look_back)

def generate_first_order_batch(data_set, look_back, gap_days):
    data_x = []
    for i in range(len(data_set) - look_back - gap_days + 1):
        a = data_set[i:(i + look_back), 0]
        data_x.append(a)
    return np.array(data_x), data_set[-look_back:, 0].reshape(1, look_back)

In [5]:
def metrics(data):
    data = scaler.inverse_transform(data)
    data = 2 ** (data - 1)
    print(data)
    
    a = data[:, 0]
    b = data[:, 1]
    res = np.sqrt(np.sum((a - b) ** 2) / len(a)) / np.mean(a)
    print('validate rmse:', 1 - res)
    return (1 - res)

In [6]:
from keras.layers.advanced_activations import PReLU

def construct_lstm_model(x_train, x_test, y_train, y_test, test, car_name, batch_size):
    print('batch_size:', batch_size)
    input1_ = Input(shape=(1, look_back), name='input1', dtype='float32')
    input2_ = Input(shape=(1, look_back), name='input2', dtype='float32')

    seq1 = LSTM(32, return_sequences=True, kernel_initializer=glorot_uniform(seed=seed))(input1_)    # , input_shape=(1, look_back)   GRU   LSTM
    seq1 = LSTM(32, return_sequences=False, kernel_initializer=glorot_uniform(seed=seed))(seq1)

    seq2 = LSTM(32, return_sequences=True, kernel_initializer=glorot_uniform(seed=seed))(input2_)
    seq2 = LSTM(32, return_sequences=False, kernel_initializer=glorot_uniform(seed=seed))(seq2)

    merged = concatenate([seq1, seq2])
    merged = BatchNormalization()(merged)
    merged = Dropout(0.5)(merged)
    output_ = Dense(units=1, kernel_initializer=glorot_uniform(seed=seed))(merged)
    output_ = PReLU()(output_)
    model = Model(inputs=[input1_, input2_], outputs=[output_])
    model.compile(loss='mean_squared_error', optimizer='adam')              # AdamW(weight_decay=0.08)   'adam'
#     model.summary()
    early_stopping = EarlyStopping('val_loss', patience=20, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=0)
    best_model_path = './model/' + car_name + '.h5'
    checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', mode='min', save_best_only=True, verbose=0, save_weights_only=False)
    
    callbacks = [early_stopping, reduce_lr]         # checkpoint   # reduce_lr
    model.fit([x_train[:, :, :look_back], x_train[:, :, look_back:]], y_train, epochs=500, batch_size=batch_size, verbose=0, shuffle=False, callbacks=callbacks, validation_data=([x_test[:, :, :look_back], x_test[:, :, look_back:]], y_test))
#     model.load_weights(best_model_path)     # load best model parameters   shuffle=False
    predict = model.predict([test[:, :, :look_back], test[:, :, look_back:]])
    predict = scaler.inverse_transform(predict)
    predict = 2 ** (predict - 1)
#     print(np.hstack((y_test, model.predict(x_test))))
    rmse = metrics(np.hstack((y_test, model.predict([x_test[:, :, :look_back], x_test[:, :, look_back:]]))))
    return predict, rmse

In [7]:
file_name = os.listdir('./model/')

look_back = 9     # 10  9
label_length = 4
batch_size = 16
submit = pd.DataFrame()
flag = 0
total = []
thresh = 1.2368

for car in cars:
    labels = np.zeros([len(provinces), label_length])
    rmse_total = 0
    for k in range(1, 5, 1):
        print('{} car:{}'.format(flag+1, car))
        train = np.array([[0 for _ in range(look_back)]])
        train_feature = np.array([[0 for _ in range(look_back)]])
        label = np.array([[0]])
        test = np.array([[0 for _ in range(look_back)]])
        test_feature = np.array([[0 for _ in range(look_back)]])

        for province in provinces: 
            temp = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
    #         print(temp[['province', 'model', 'regYear', 'regMonth', 'salesVolume']])
            temp['first_order'] = temp['salesVolume'].diff(1)

            data = temp['salesVolume'].values.reshape(-1, 1).astype('float32')[1:]
            data_b = temp['first_order'].values.reshape(-1, 1).astype('float32')[1:]

            train_data, train_label, test_data = generate_batch(data, look_back, k)
            train_data_b, test_data_b = generate_first_order_batch(data_b, look_back, k)

            train = np.vstack((train, train_data))
            label = np.vstack((label, train_label))
            test = np.vstack((test, test_data))
            train_feature = np.vstack((train_feature, train_data_b))
            test_feature = np.vstack((test_feature, test_data_b))

        train = np.hstack((train, train_feature))
        test = np.hstack((test, test_feature))
        train = train[1:]
        label = label[1:]
        test = test[1:]
        train = np.reshape(train, (-1, 1, look_back * 2))
        test = np.reshape(test, (-1, 1, look_back * 2))
        x_train,x_test,y_train,y_test = train_test_split(train, label, test_size=0.1, random_state=seed)

        print('train shape:', train.shape)
        print('label shape:', label.shape)
        print('test shape:', test.shape)
        res, rmse = construct_lstm_model(x_train, x_test, y_train, y_test, test, car, int((train.shape[0] * 0.9) / 22))

        K.clear_session()    #  清除tf session

        labels[:, k-1] = res.reshape(1, -1)
        rmse_total += rmse
    print(labels)
    print('average rmse:', rmse_total / 4)
    flag += 1
    total.append(rmse_total / 4)

    index = 0
    submit_partial = pd.DataFrame()
    for province_2 in provinces: 
        temp = evaluation_public[(evaluation_public['model'] == car) & (evaluation_public['province'] == province_2)]
        temp['forecastVolum'] = labels[index]
        submit_partial = pd.concat([submit_partial, temp], axis=0)
        index += 1
    submit = pd.concat([submit, submit_partial], axis=0)

print('all validation rmse:', np.mean(total))  # 0.6555167883490066
car_rmse = pd.DataFrame({'cars': cars, 'rmse': total})
print(car_rmse)
# car_rmse.to_csv('car_rmse.csv', encoding='utf-8', index=None)

[]
1 car:a50a6db3d691e9ce
train shape: (308, 1, 18)
label shape: (308, 1)
test shape: (22, 1, 18)
batch_size: 12
Epoch 00050: early stopping
[[505.99990097 409.2824898 ]
 [205.00005415 201.67704044]
 [277.00001148 240.36235554]
 [186.99997475 219.29153903]
 [ 69.9999972   59.92313646]
 [108.00001109  86.58537882]
 [438.99998416 409.88593771]
 [456.99998136 349.27470109]
 [129.00003593 116.08624604]
 [215.9999768  150.45722625]
 [291.00005581 226.7024095 ]
 [146.99999695 121.21439178]
 [155.99997376 188.55668893]
 [326.00008753 282.558394  ]
 [469.00003486 451.90698109]
 [279.0000113  295.28760309]
 [250.9999714  199.67085678]
 [179.99997496 153.68630964]
 [174.00002648 103.26565305]
 [245.00000606 331.12364337]
 [208.99994294 145.75790427]
 [ 76.00000245  63.13906294]
 [ 45.99999794  48.67378922]
 [145.99997893 160.83667871]
 [146.99999695 258.20823935]
 [285.99997371 301.84841122]
 [551.00014181 421.08858791]
 [246.99996538 179.70637813]
 [ 96.00000781  87.54404892]
 [263.99994088 237

Epoch 00057: early stopping
[[ 146.99999695  173.02471397]
 [ 233.00000251  169.00863355]
 [1020.00018486  686.49030389]
 [ 371.00004081  262.91748685]
 [  76.00000245  109.83341225]
 [ 179.99997496   93.65728221]
 [ 384.00009048  297.84498664]
 [ 407.99997673  374.62757278]
 [ 233.00000251  180.55967038]
 [ 308.00004471  251.06976067]
 [ 121.99999587  114.91553564]
 [  93.00000537   75.64112263]
 [ 464.00005273  245.96644549]
 [ 145.00001747  140.03877062]
 [1803.00025245  678.04050985]
 [ 182.99997653  185.63430761]
 [ 120.99999821  127.28442535]
 [  69.9999972    82.66484159]
 [ 336.00004907  180.49558759]
 [  67.99999205  141.37797993]
 [ 479.00013488  387.31087643]
 [  73.0000048   118.20674048]
 [ 195.99997326  244.23102017]
 [  92.00000298  110.89634338]
 [  78.00000326   79.18109566]]
validate rmse: 0.20104950857465576
[[ 404.56689453  416.14266968  390.74911499  364.59738159]
 [ 196.762146    230.66497803  218.86842346  203.6809082 ]
 [ 443.93676758  463.61575317  430.15350342

train shape: (286, 1, 18)
label shape: (286, 1)
test shape: (22, 1, 18)
batch_size: 11
Epoch 00056: early stopping
[[1001.99976533 1063.26713459]
 [ 556.0001412   365.80809355]
 [ 461.00000027  459.19683789]
 [ 164.99997691  235.52132089]
 [ 311.99997159  297.31391469]
 [1631.00024893 1075.42744594]
 [1131.00006436  995.49547442]
 [ 182.00003656  196.11731783]
 [1283.99969223 1077.84915299]
 [ 140.99998243  132.33262144]
 [ 650.99994295  541.83505937]
 [ 860.99981585  803.71124431]
 [ 680.99984896  625.77382606]
 [1262.99983346 1326.7371733 ]
 [ 388.999972    361.20421521]
 [ 272.99995068  309.09868397]
 [ 705.00013454  911.57096309]
 [ 425.00003076  373.06902176]
 [ 140.99998243  118.72197732]
 [ 451.99991752  408.31741331]
 [ 260.99994013  334.32382677]
 [  74.00000482  125.24454609]
 [ 856.00012144  761.51706123]
 [ 150.9999799   110.37214058]
 [1244.00006883 1071.7472152 ]
 [1111.99972946  693.12855333]
 [ 426.99989052  370.90479738]
 [ 737.99999366  736.28560814]
 [1590.00008955 1

train shape: (308, 1, 18)
label shape: (308, 1)
test shape: (22, 1, 18)
batch_size: 12
Epoch 00057: early stopping
[[ 755.99996772  391.99772516]
 [ 104.99998589   98.17508626]
 [ 909.99994715  555.91647447]
 [ 453.000062    368.12966546]
 [ 243.99994048  128.43148857]
 [  68.9999904    68.32775963]
 [ 414.99992771  388.47613727]
 [ 777.0000875   308.06745599]
 [  21.00000262   20.15582583]
 [ 201.00004069  113.57186674]
 [ 362.9999884   226.70488363]
 [  44.99999972   21.55663348]
 [ 515.99992695  352.81178475]
 [  66.99999548   56.27969293]
 [ 169.9999718   138.16349562]
 [ 532.99996301  425.54818278]
 [ 229.00005252  100.18255735]
 [ 136.00003364   93.33503434]
 [ 325.00001247  298.06524817]
 [  21.00000262   23.93339689]
 [ 134.99999684   69.58408468]
 [  82.00000212   81.20748584]
 [  65.00000066   78.48084548]
 [ 112.9999944   141.86117066]
 [2477.99970042  364.34357116]
 [  26.99999861   40.84511943]
 [  76.00000245   46.94504877]
 [ 314.99995226  187.43226793]
 [ 133.00000263  

Epoch 00042: early stopping
[[ 644.00011222  627.19882938]
 [ 239.99993885  286.58358543]
 [ 879.99991082  667.14807263]
 [ 252.00006595  211.1190192 ]
 [ 157.00002479  133.17796751]
 [ 355.00009949  271.79315565]
 [ 507.00011852  460.47388773]
 [ 317.9999176   155.50970859]
 [ 207.99994095  176.62345915]
 [ 501.00013178  474.16473225]
 [ 199.00001997  104.93369458]
 [ 118.00000492   93.65539937]
 [1856.99967752 1664.14902268]
 [ 472.0000925   336.51555081]
 [1397.00031787  983.14905031]
 [ 909.99994715  913.85567091]
 [ 459.00011542  299.69013204]
 [ 220.99999952  167.4256741 ]
 [ 488.99994471  567.12739693]
 [ 308.99998517  350.52741298]
 [ 315.99992899  195.88697448]
 [ 121.99999587  222.20502232]
 [ 888.00017958  724.15752489]
 [ 109.99999498  111.68472123]
 [ 104.99998589   98.27017949]]
validate rmse: 0.7336624994164296
[[1342.09155273  784.7668457   909.66296387  902.046875  ]
 [ 343.70654297  230.21864319  181.53388977  161.78077698]
 [ 832.3480835   568.92541504  611.48248291 

...

76 car:63065128401bb3ff
train shape: (308, 1, 18)
label shape: (308, 1)
test shape: (22, 1, 18)
batch_size: 12
Epoch 00038: early stopping
[[2533.00006227 2259.57974589]
 [1810.00017269 1484.95368116]
 [2119.00036708 1648.67599554]
 [1800.99948853 1985.10564363]
 [ 826.99994548  679.30002947]
 [ 882.99989861 1469.95326331]
 [4988.99985057 3143.76291486]
 [7066.00146145 4452.83098846]
 [ 275.99992493  254.73430186]
 [2108.00051678 1262.48486294]
 [2723.0001902  1707.34263958]
 [ 466.00004098  343.49225294]
 [1398.99979179 1546.45116712]
 [1235.00021645  591.42018014]
 [1358.99977231 1352.82679735]
 [2053.00029252 1938.76536867]
 [2760.99926852 2364.04561853]
 [ 365.9999813   272.10775549]
 [1576.99994121 1225.20307943]
 [ 976.99998808  917.68819721]
 [ 407.00005056  229.11385949]
 [ 749.00003914  515.08046597]
 [ 413.00004393  346.06873572]
 [1307.0000319  1267.46929282]
 [1756.0002076  1034.56033034]
 [ 586.99985925  371.29146277]
 [2455.00049599 1714.367978  ]
 [2357.00053167 2118.150

Epoch 00032: early stopping
[[ 203.99997262  137.51601008]
 [ 274.00002607  150.97838484]
 [1114.00009728  541.0578626 ]
 [ 432.9998972   250.70343933]
 [ 136.00003364  218.5987245 ]
 [ 249.99996866  382.45043385]
 [ 635.00007591  573.8784492 ]
 [ 781.99997747  712.77966679]
 [ 234.00000576   92.2576614 ]
 [ 509.00005687  754.7369477 ]
 [ 283.00003144  449.94344174]
 [ 400.99991886  314.42709449]
 [ 900.99998566  160.83806447]
 [ 201.00004069  150.11779994]
 [1908.99992917  527.54580511]
 [ 312.99995326  165.81877157]
 [  46.99999495   58.2652172 ]
 [  52.00000708  107.82499871]
 [ 177.00004151  307.86190655]
 [ 196.99998379   89.56483034]
 [ 851.99984214  672.21749879]
 [  34.00000316   63.92908314]
 [ 298.99995814  138.50834842]
 [  65.99999399  107.41855101]
 [ 176.00002774  140.97228677]]
validate rmse: 0.159341986770123
[[ 493.87817383  490.21444702  154.66792297  146.91902161]
 [ 453.44876099  396.04653931  123.62921143   36.69394684]
 [ 622.93261719  723.24499512  156.62173462  

Epoch 00032: early stopping
[[106.0000048  215.16339149]
 [ 54.00000138  39.18552955]
 [ 38.99999862  21.76131993]
 [ 99.00001008 111.23250903]
 [ 18.99999768  40.02888982]
 [142.00000596 157.35338537]
 [298.99995814 148.69042156]
 [ 88.00000708  53.32111933]
 [134.00000129 146.62835897]
 [ 13.99999905  19.60782684]
 [122.99999157 172.14344025]
 [ 56.99999206  38.94068025]
 [134.99999684 163.2319206 ]
 [290.00005731 151.50587381]
 [588.99997224 299.20612143]
 [111.99998612  54.65053594]
 [185.000003   181.78303732]
 [ 33.00000393  23.82117452]
 [ 80.00000397  91.35970918]
 [ 34.9999959   26.1125498 ]
 [ 43.00000037  55.12862656]
 [120.99999821  83.60788081]
 [109.99999498  76.61260858]
 [ 32.00000315  19.10833783]
 [210.0000483  176.82840817]
 [223.99998952 148.40185625]
 [483.99992854 328.1212233 ]
 [ 63.99999286  34.22898122]
 [358.99992633 186.20072003]]
validate rmse: 0.4203678173049673
79 car:32d3069d17aa47c2
train shape: (264, 1, 18)
label shape: (264, 1)
test shape: (22, 1, 18)


train shape: (308, 1, 18)
label shape: (308, 1)
test shape: (22, 1, 18)
batch_size: 12
Epoch 00055: early stopping
[[190.99994644 184.29756183]
 [126.99997512 134.32034242]
 [301.00004527 323.7600432 ]
 [287.99993575 301.23682802]
 [ 86.00000737  97.26439087]
 [287.00002598 255.13904504]
 [214.99994616 167.7949819 ]
 [263.99994088 201.38382622]
 [ 78.00000326  54.46409455]
 [190.0000514  158.25663165]
 [193.9999998  155.96628579]
 [ 52.99999831  62.96504196]
 [351.00007632 337.26713593]
 [283.00003144 215.40761705]
 [737.99999366 565.18207981]
 [298.00002557 284.604902  ]
 [232.00000846 174.96023377]
 [102.99999683 106.19425568]
 [138.00003108 132.11829227]
 [129.00003593 117.15675129]
 [103.99999232  91.66444819]
 [102.99999683  83.88485961]
 [110.99999676  76.66730526]
 [168.99999388 137.64316094]
 [190.0000514  119.5648212 ]
 [124.99997469 116.08347889]
 [332.00007938 229.13623282]
 [138.00003108 133.31757028]
 [289.0000092  258.15277623]
 [270.99999632 345.12681106]
 [717.99990806 

Epoch 00056: early stopping
[[ 543.99990604  568.95684465]
 [ 475.99987513  415.38889646]
 [ 668.99998704  478.99683328]
 [ 302.99993598  296.02225345]
 [ 381.00002823  349.12025707]
 [ 616.00013695  585.28752154]
 [ 792.00003647  676.4657997 ]
 [ 848.99983586  605.66654767]
 [ 341.99997286  384.38119875]
 [ 656.99983187  668.91506897]
 [ 245.99993147  255.34517775]
 [ 219.00001062  181.98038418]
 [ 819.99987211  673.52268185]
 [ 315.99992899  282.68387946]
 [ 891.99987953  509.40221914]
 [ 234.00000576  183.29800674]
 [ 586.99985925  539.75384997]
 [ 230.00006226  196.73198783]
 [1547.00021605 1213.75902164]
 [ 338.00001384  412.3372968 ]
 [1094.00017903  756.35308828]
 [ 555.00015884  791.30613358]
 [ 425.00003076  409.93255662]
 [ 250.9999714   258.87651139]
 [ 397.0000238   456.47476306]]
validate rmse: 0.7228413970232987
[[ 634.88287354  538.02502441  441.21307373  373.74804688]
 [ 389.39849854  361.04327393  283.27883911  230.66421509]
 [ 688.0668335   591.35510254  568.17529297 

```
all validation rmse: 0.611178786215723
```

In [8]:
submit.sort_values(by=['id'], ascending=True)['forecastVolum'].mean()

471.3008293454306

In [9]:
submit = submit.sort_values(by=['id'], ascending=True)
submit['forecastVolum'] = submit['forecastVolum'].map(lambda index: int(np.round(index)))
print('res store over')

res store over


In [10]:
sub = submit.sort_values(by=['id'], ascending=True)
print(len(sub))
sub.describe()
sub.groupby(['regMonth'])['forecastVolum'].mean()

7216


,id,adcode,regYear,regMonth,forecastVolum
count,7216.000000,7216.000000,7216.0,7216.000000,7216.000000
mean,3658.939024,347727.272727,2018.0,2.500000,471.302522
std,2114.152561,136288.963363,0.0,1.118111,618.115307
min,1.000000,110000.000000,2018.0,1.000000,1.000000
25%,1826.750000,230000.000000,2018.0,1.750000,119.000000
50%,3652.500000,355000.000000,2018.0,2.500000,275.000000
75%,5500.250000,440000.000000,2018.0,3.250000,565.250000
max,7304.000000,610000.000000,2018.0,4.000000,8105.000000


regMonth
1    590.483925
2    462.738914
3    430.183481
4    401.803769
Name: forecastVolum, dtype: float64

In [11]:
sub = submit.copy()
sub['forecastVolum'] = sub['forecastVolum'] / thresh
sub['forecastVolum'] = sub['forecastVolum'].map(lambda index: int(np.round(index)))
sub.describe()
sub.groupby(['regMonth'])['forecastVolum'].mean()
sub[['id', 'forecastVolum']].to_csv('./submit/fusai_lstm_2.csv', encoding='utf-8', index=None)

,id,adcode,regYear,regMonth,forecastVolum
count,7216.000000,7216.000000,7216.0,7216.000000,7216.000000
mean,3658.939024,347727.272727,2018.0,2.500000,381.067489
std,2114.152561,136288.963363,0.0,1.118111,499.773405
min,1.000000,110000.000000,2018.0,1.000000,1.000000
25%,1826.750000,230000.000000,2018.0,1.750000,96.000000
50%,3652.500000,355000.000000,2018.0,2.500000,222.000000
75%,5500.250000,440000.000000,2018.0,3.250000,457.250000
max,7304.000000,610000.000000,2018.0,4.000000,6553.000000


regMonth
1    477.431264
2    374.138027
3    347.820399
4    324.880266
Name: forecastVolum, dtype: float64